In [95]:
import pandas as pd
from sklearn import preprocessing
import numpy as np

from gnnad.graphanomaly import GNNAD
from gnnad.plot import plot_test_anomalies, plot_predictions, plot_sensor_error_scores

import datetime as dt
from datetime import datetime


def normalise(X, scaler_fn):
    scaler = scaler_fn.fit(X)
    return pd.DataFrame(scaler.transform(X), index=X.index, columns = X.columns)

In [96]:
df = pd.read_csv('/nfs/home/canzen/gnnad/kungsbacka_data/bus_383_signals.csv', parse_dates=['datetime'])
df.head()

,t_ms,datetime,GPS0,GPS1,EngineSpeed,FuelRate,VehicleSpeed,BrakePedalPos,ABSActive,Wet Tank Air Pressure,RelSpdFrontRight,RelSpdFrontLeft,AcceleratorPedalPos,Total Vehicle Distance,Selected Gear,AmbientAirTemperature,EngineTorqueMode,ActEngPercTorque,lon,lat
0,2.128352e+10,2012-02-17 13:28:25,1.003055,0.209433,582.875,9.167785,1.699110,0.0,0.0,10.13565,0.0000,0.0000,32.8,NaN,4.0,3.46875,15.0,36.0,57.470841,11.999608
1,2.128353e+10,2012-02-17 13:28:27,1.003047,0.209441,830.875,10.173284,6.597234,0.0,0.0,10.13565,0.0625,-0.0625,58.4,NaN,4.0,3.46875,15.0,49.0,57.470368,12.000092
2,2.128353e+10,2012-02-17 13:28:29,1.003045,0.209443,1475.750,32.944879,12.596850,0.0,0.0,10.27355,0.0625,-0.0625,78.0,NaN,4.0,NaN,15.0,55.0,57.470254,12.000203
3,2.128353e+10,2012-02-17 13:28:31,1.003044,0.209444,925.875,0.000000,16.295832,0.0,0.0,10.48040,0.0000,0.0000,91.2,NaN,4.0,3.46875,15.0,3.0,57.470213,12.000243
4,2.128353e+10,2012-02-17 13:28:33,1.003044,0.209444,1170.500,38.859579,20.795544,0.0,0.0,10.54935,-0.1250,0.1250,92.0,NaN,7.0,3.46875,15.0,84.0,57.470199,12.000255


In [97]:
df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time
df['label'] = np.zeros(len(df))

In [98]:
df

,t_ms,datetime,GPS0,GPS1,EngineSpeed,FuelRate,VehicleSpeed,BrakePedalPos,ABSActive,Wet Tank Air Pressure,...,Total Vehicle Distance,Selected Gear,AmbientAirTemperature,EngineTorqueMode,ActEngPercTorque,lon,lat,date,time,label
0,2.128352e+10,2012-02-17 13:28:25,1.003055,0.209433,582.875,9.167785,1.699110,0.0,0.0,10.13565,...,NaN,4.0,3.46875,15.0,36.0,57.470841,11.999608,2012-02-17,13:28:25,0.0
1,2.128353e+10,2012-02-17 13:28:27,1.003047,0.209441,830.875,10.173284,6.597234,0.0,0.0,10.13565,...,NaN,4.0,3.46875,15.0,49.0,57.470368,12.000092,2012-02-17,13:28:27,0.0
2,2.128353e+10,2012-02-17 13:28:29,1.003045,0.209443,1475.750,32.944879,12.596850,0.0,0.0,10.27355,...,NaN,4.0,NaN,15.0,55.0,57.470254,12.000203,2012-02-17,13:28:29,0.0
3,2.128353e+10,2012-02-17 13:28:31,1.003044,0.209444,925.875,0.000000,16.295832,0.0,0.0,10.48040,...,NaN,4.0,3.46875,15.0,3.0,57.470213,12.000243,2012-02-17,13:28:31,0.0
4,2.128353e+10,2012-02-17 13:28:33,1.003044,0.209444,1170.500,38.859579,20.795544,0.0,0.0,10.54935,...,NaN,7.0,3.46875,15.0,84.0,57.470199,12.000255,2012-02-17,13:28:33,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,2.365421e+10,2012-03-15 23:59:51,1.003278,0.208744,1329.750,13.308075,51.395148,0.0,0.0,12.41100,...,NaN,10.0,5.00000,15.0,26.0,57.483582,11.960132,2012-03-15,23:59:51,0.0
102672,2.365421e+10,2012-03-15 23:59:53,1.003279,0.208736,1338.875,16.147131,51.695910,0.0,0.0,12.61785,...,NaN,10.0,5.00000,15.0,29.0,57.483653,11.959687,2012-03-15,23:59:53,0.0
102673,2.365422e+10,2012-03-15 23:59:55,1.003280,0.208728,1365.000,15.969690,52.895052,0.0,0.0,12.82470,...,NaN,10.0,5.00000,15.0,29.0,57.483732,11.959245,2012-03-15,23:59:55,0.0
102674,2.365422e+10,2012-03-15 23:59:57,1.003281,0.208720,1384.875,15.733102,53.594226,0.0,0.0,13.03155,...,NaN,10.0,5.00000,15.0,28.0,57.483771,11.958801,2012-03-15,23:59:57,0.0


In [99]:
def anomaly_labels(df, start, end):
    mask = (df['Date'] >= start) & (df['Date'] <= end)
    df.loc[mask, 'label'] = 1
    return df

In [100]:
start1 = datetime.strptime('2012-02-27', "%Y-%m-%d").date
end1 = datetime.strptime('2012-02-27', "%Y-%m-%d").date

start2 = datetime.strptime('2012-03-05', "%Y-%m-%d").date
end2 = datetime.strptime('2012-03-08', "%Y-%m-%d").date

In [101]:
from datetime import date

In [102]:
df.loc[df['date'] == date(2012, 2, 27), 'label'] = 1
df.loc[df['date'] == date(2012, 3, 5), 'label'] = 1
df.loc[df['date'] == date(2012, 3, 6), 'label'] = 1
df.loc[df['date'] == date(2012, 3, 7), 'label'] = 1
df.loc[df['date'] == date(2012, 3, 8), 'label'] = 1

In [103]:
df.isna().sum()

t_ms                          0
datetime                      0
GPS0                          0
GPS1                          0
EngineSpeed                 226
FuelRate                    493
VehicleSpeed                224
BrakePedalPos               475
ABSActive                   475
Wet Tank Air Pressure      5589
RelSpdFrontRight            473
RelSpdFrontLeft             473
AcceleratorPedalPos         333
Total Vehicle Distance    87142
Selected Gear             10376
AmbientAirTemperature      4253
EngineTorqueMode            225
ActEngPercTorque            225
lon                           0
lat                           0
date                          0
time                          0
label                         0
dtype: int64

In [104]:
df.drop(['Total Vehicle Distance', 'datetime', 't_ms', 'time'], inplace=True, axis=1)

In [105]:
df = df.fillna(df.mean())

/tmp/ipykernel_1732146/114435927.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df = df.fillna(df.mean())


 # bus 383  15 rows
['383', '2012-02-27', '2012-02-27', 1, [6], u'Volvo Action Service'] ,
['383', '2012-03-05', '2012-03-08', 1, [4], u'Bus towed'] ,
['383', '2012-06-11', '2012-06-15', 1, [4], u'Bus towed'] ,
['383', '2012-07-30', '2012-08-01', 1, [4, 8], u'Regulator + condensor congested'] ,
['383', '2012-10-18', '2012-10-18', 0, [4], ''] ,
['383', '2012-11-15', '2012-11-19', 1, [3], u'Towing back to Kungsbacka?'] ,
['383', '2013-09-27', '2013-09-30', 1, [8], u'4-circuit valve'] ,
['383', '2014-04-01', '2014-04-03', 1, [6], u'4-circuit valve'] ,
['383', '2014-08-22', '2014-08-26', 1, [8], u'4-circuit valve'] ,
['383', '2014-08-28', '2014-09-01', 1, [2], u'Bus towed but there is also a footbrake valve failure.'] ,
['383', '2015-04-28', '2015-04-28', 1, [7], u'Customer complaint on leak'] ,

In [107]:
df.loc[df['date'] >= date(2012, 2, 27)]

,GPS0,GPS1,EngineSpeed,FuelRate,VehicleSpeed,BrakePedalPos,ABSActive,Wet Tank Air Pressure,RelSpdFrontRight,RelSpdFrontLeft,AcceleratorPedalPos,Selected Gear,AmbientAirTemperature,EngineTorqueMode,ActEngPercTorque,lon,lat,date,label
24729,1.000512,0.212470,1173.750,0.000000,73.995264,1.6,0.0,10.480400,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.325094,12.173617,2012-02-27,1.0
24730,1.000494,0.212483,1131.875,0.000000,71.792280,12.4,0.0,11.344071,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324070,12.174368,2012-02-27,1.0
24731,1.000488,0.212476,1066.875,0.000000,67.792536,15.6,0.0,10.411450,0.0625,-0.0625,0.0,12.0,-5.684342e-14,15.0,0.0,57.323753,12.173953,2012-02-27,1.0
24732,1.000494,0.212473,957.125,0.000000,60.996096,18.8,0.0,10.411450,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324073,12.173831,2012-02-27,1.0
24733,1.000500,0.212472,844.750,0.000000,54.195750,16.0,0.0,11.344071,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324399,12.173767,2012-02-27,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102671,1.003278,0.208744,1329.750,13.308075,51.395148,0.0,0.0,12.411000,0.0000,0.0000,56.0,10.0,5.000000e+00,15.0,26.0,57.483582,11.960132,2012-03-15,0.0
102672,1.003279,0.208736,1338.875,16.147131,51.695910,0.0,0.0,12.617850,0.0000,0.0000,59.6,10.0,5.000000e+00,15.0,29.0,57.483653,11.959687,2012-03-15,0.0
102673,1.003280,0.208728,1365.000,15.969690,52.895052,0.0,0.0,12.824700,0.0000,0.0000,59.6,10.0,5.000000e+00,15.0,29.0,57.483732,11.959245,2012-03-15,0.0
102674,1.003281,0.208720,1384.875,15.733102,53.594226,0.0,0.0,13.031550,0.0000,0.0000,59.6,10.0,5.000000e+00,15.0,28.0,57.483771,11.958801,2012-03-15,0.0


In [111]:
train = df.loc[df['date'] < date(2012, 2, 27)]
X_train = train.iloc[:, :-1]
X_train.index = X_train.date
X_train.drop('date', inplace=True, axis=1)

In [112]:
X_train

,GPS0,GPS1,EngineSpeed,FuelRate,VehicleSpeed,BrakePedalPos,ABSActive,Wet Tank Air Pressure,RelSpdFrontRight,RelSpdFrontLeft,AcceleratorPedalPos,Selected Gear,AmbientAirTemperature,EngineTorqueMode,ActEngPercTorque,lon,lat
date,,,,,,,,,,,,,,,,,
2012-02-17,1.003055,0.209433,582.875,9.167785,1.699110,0.0,0.0,10.13565,0.0000,0.0000,32.8,4.0,3.468750,15.0,36.0,57.470841,11.999608
2012-02-17,1.003047,0.209441,830.875,10.173284,6.597234,0.0,0.0,10.13565,0.0625,-0.0625,58.4,4.0,3.468750,15.0,49.0,57.470368,12.000092
2012-02-17,1.003045,0.209443,1475.750,32.944879,12.596850,0.0,0.0,10.27355,0.0625,-0.0625,78.0,4.0,7.847516,15.0,55.0,57.470254,12.000203
2012-02-17,1.003044,0.209444,925.875,0.000000,16.295832,0.0,0.0,10.48040,0.0000,0.0000,91.2,4.0,3.468750,15.0,3.0,57.470213,12.000243
2012-02-17,1.003044,0.209444,1170.500,38.859579,20.795544,0.0,0.0,10.54935,-0.1250,0.1250,92.0,7.0,3.468750,15.0,84.0,57.470199,12.000255
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-02-26,1.002458,0.212101,602.375,3.903702,0.000000,0.0,0.0,7.37765,0.0000,0.0000,0.0,0.0,-1.125000,15.0,15.0,57.436622,12.152465
2012-02-26,1.002458,0.212100,598.625,3.903702,0.000000,0.0,0.0,7.37765,0.0000,0.0000,0.0,0.0,-1.125000,15.0,16.0,57.436626,12.152463
2012-02-26,1.002458,0.212101,598.875,3.844555,0.000000,0.0,0.0,7.37765,0.0000,0.0000,0.0,0.0,-1.125000,15.0,16.0,57.436623,12.152465


In [113]:
test = df.loc[df['date'] >= date(2012, 2, 27)]
X_test = test.iloc[:, :-1]
X_test.index = X_test.date
X_test.drop('date', inplace=True, axis=1)
y_test = test['label']

In [114]:
X_test

,GPS0,GPS1,EngineSpeed,FuelRate,VehicleSpeed,BrakePedalPos,ABSActive,Wet Tank Air Pressure,RelSpdFrontRight,RelSpdFrontLeft,AcceleratorPedalPos,Selected Gear,AmbientAirTemperature,EngineTorqueMode,ActEngPercTorque,lon,lat
date,,,,,,,,,,,,,,,,,
2012-02-27,1.000512,0.212470,1173.750,0.000000,73.995264,1.6,0.0,10.480400,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.325094,12.173617
2012-02-27,1.000494,0.212483,1131.875,0.000000,71.792280,12.4,0.0,11.344071,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324070,12.174368
2012-02-27,1.000488,0.212476,1066.875,0.000000,67.792536,15.6,0.0,10.411450,0.0625,-0.0625,0.0,12.0,-5.684342e-14,15.0,0.0,57.323753,12.173953
2012-02-27,1.000494,0.212473,957.125,0.000000,60.996096,18.8,0.0,10.411450,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324073,12.173831
2012-02-27,1.000500,0.212472,844.750,0.000000,54.195750,16.0,0.0,11.344071,0.0000,0.0000,0.0,12.0,-5.684342e-14,15.0,0.0,57.324399,12.173767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-03-15,1.003278,0.208744,1329.750,13.308075,51.395148,0.0,0.0,12.411000,0.0000,0.0000,56.0,10.0,5.000000e+00,15.0,26.0,57.483582,11.960132
2012-03-15,1.003279,0.208736,1338.875,16.147131,51.695910,0.0,0.0,12.617850,0.0000,0.0000,59.6,10.0,5.000000e+00,15.0,29.0,57.483653,11.959687
2012-03-15,1.003280,0.208728,1365.000,15.969690,52.895052,0.0,0.0,12.824700,0.0000,0.0000,59.6,10.0,5.000000e+00,15.0,29.0,57.483732,11.959245


In [116]:
X_test = normalise(X_test, preprocessing.StandardScaler())
X_train = normalise(X_train, preprocessing.StandardScaler())

In [130]:
# run model
model = GNNAD(threshold_type="max_validation", topk=5, 
              slide_win=20, epoch=50, early_stop_win=10, 
              slide_stride=5, embed_dim=64, 
              out_layer_inter_dim=64, device='cpu')
fitted_model = model.fit(X_train, X_test, y_test.values)

epoch (0 / 50) (Loss:0.88692601, ACU_loss:31.04241043)
epoch (1 / 50) (Loss:0.75335047, ACU_loss:26.36726636)
epoch (2 / 50) (Loss:0.73254021, ACU_loss:25.63890743)
epoch (3 / 50) (Loss:0.71433762, ACU_loss:25.00181654)
epoch (4 / 50) (Loss:0.68267473, ACU_loss:23.89361557)
epoch (5 / 50) (Loss:0.65368025, ACU_loss:22.87880862)
epoch (6 / 50) (Loss:0.66265003, ACU_loss:23.19275120)
epoch (7 / 50) (Loss:0.65946659, ACU_loss:23.08133069)
epoch (8 / 50) (Loss:0.61888456, ACU_loss:21.66095975)
epoch (9 / 50) (Loss:0.61099256, ACU_loss:21.38473958)
epoch (10 / 50) (Loss:0.61659270, ACU_loss:21.58074465)
epoch (11 / 50) (Loss:0.61539310, ACU_loss:21.53875834)
epoch (12 / 50) (Loss:0.65407184, ACU_loss:22.89251450)
epoch (13 / 50) (Loss:0.64239838, ACU_loss:22.48394331)
epoch (14 / 50) (Loss:0.64276995, ACU_loss:22.49694827)
epoch (15 / 50) (Loss:0.61939536, ACU_loss:21.67883775)
epoch (16 / 50) (Loss:0.60089187, ACU_loss:21.03121558)
epoch (17 / 50) (Loss:0.60227985, ACU_loss:21.07979476)
ep

In [3]:
# read in training data
train = pd.read_csv('/nfs/home/canzen/gnnad/kungsbacka_data/bus_383_signals.csv', index_col=0)
X_train = train.iloc[:, :-1]
#X_train.index = pd.to_datetime(X_train.index)

# read in test data
test = pd.read_csv('/nfs/home/canzen/gnnad/swat_data/test.csv', index_col=0)
#X_tmp.index = pd.to_datetime(X_tmp.index)
X_test = test.iloc[:, :-1]
y_test = test['attack']

# normalise
#X_test = normalise(X_test, preprocessing.StandardScaler())
#X_train = normalise(X_train, preprocessing.StandardScaler())

# create ANOOMS dict for plotting
#ANOMS = {'type1': {}}
#X_test_anoms = X_tmp.iloc[:,8:]

#for i in range(len(X_test_anoms.columns)):
#    anom_col_name = X_test_anoms.columns[i]
#    sensor_col_name = X_test.columns[i]
#    anom_idxs = X_test_anoms[anom_col_name][X_test_anoms[anom_col_name]].index
#
#    if len(anom_idxs) > 0:
#        ANOMS['type1'][sensor_col_name] = anom_idxs

In [4]:
# plot input data
#plot_test_anomalies(X_test, ANOMS)

In [7]:
# model summary
fitted_model.summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Embedding-1                   [-1, 64]           3,200
         Embedding-2                   [-1, 64]           3,200
            Linear-3                   [-1, 64]             320
    SumAggregation-4                [-1, 1, 64]               0
        GraphLayer-5                   [-1, 64]              64
       BatchNorm1d-6                   [-1, 64]             128
              ReLU-7                   [-1, 64]               0
          GNNLayer-8                   [-1, 64]               0
         Embedding-9                   [-1, 64]           3,200
      BatchNorm1d-10               [-1, 64, 50]             128
          Dropout-11               [-1, 50, 64]               0
           Linear-12                [-1, 50, 1]              65
         OutLayer-13                [-1, 50, 1]               0
Total params: 10,305
Trainable params: 

In [131]:
# GDN+, sensor thresholds
preds = fitted_model.sensor_threshold_preds(tau = 99)
fitted_model.print_eval_metrics(preds)

recall: 60.6
precision: 21.3
accuracy: 76.2
specificity: 77.7
f1: 31.5


In [ ]:
# plot predictions
#plot_predictions(fitted_model, X_test, ANOMS, preds = preds, figsize=(20, 20))

In [ ]:
#plot_sensor_error_scores(fitted_model, X_test)